In [1]:
import cv2 
import pytesseract
import numpy as np
from skimage.io import imread , imshow
from matplotlib import pyplot as plt
from imutils.perspective import four_point_transform
import imutils
import scipy.misc
import pandas as pd

#%matplotlib inline


from os import listdir
from os.path import isfile, join
import argparse
from PIL import Image
import glob 

# Preprocessing + Save Image

In [2]:
def detect_angle(image):
    mask = np.zeros(image.shape, dtype=np.uint8)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    adaptive = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,15,4)

    cnts = cv2.findContours(adaptive, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    for c in cnts:
        area = cv2.contourArea(c)
        if area < 45000 and area > 20:
            cv2.drawContours(mask, [c], -1, (255,255,255), -1)

    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    h, w = mask.shape
   
    # Horizontal
    if w > h:
        left = mask[0:h, 0:0+w//2]
        right = mask[0:h, w//2:]
        left_pixels = cv2.countNonZero(left)
        right_pixels = cv2.countNonZero(right)
        return 0 if left_pixels >= right_pixels else 180
    # Vertical
    else:
        top = mask[0:h//2, 0:w]
        bottom = mask[h//2:, 0:w]
        top_pixels = cv2.countNonZero(top)
        bottom_pixels = cv2.countNonZero(bottom)
        return 90 if bottom_pixels >= top_pixels else 270



In [3]:
# Load image, grayscale, Gaussian blur, Otsu's threshold
def imgprepro(imge):
   
    folder = r'C:/Users/Chirine/Desktop/Projet DS/cin/cin'
    image = cv2.imread(os.path.join(folder,imge))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7,7), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # Find contours and sort for largest contour
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    displayCnt = None

    for c in cnts:
        # Perform contour approximation
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            displayCnt = approx
            break

    # Obtain birds' eye view of image
    warped = four_point_transform(image, displayCnt.reshape(4, 2))


    angle = detect_angle(warped)
    #print(angle)

    image = warped

    if angle == 90:
        rot = imutils.rotate_bound(image, angle= 90)
    elif angle == 270:
        rot = imutils.rotate_bound(image, angle= 270)
    elif angle == 180:
        rot = imutils.rotate_bound(image, angle= 180)
    else : 
        rot = image
    path = 'C:/Users/Chirine/Desktop/Projet DS/baseimage/images_finals/CIN/' + imge 
    plt.imsave(path,rot)

In [4]:
mypath= r'C:/Users/Chirine/Desktop/Projet DS/cin/cin'
import os
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        images.append(filename)
        #print(filename)
    return images
images = []
images = load_images_from_folder(mypath)

for n in range(0, len(images)):
    imgprepro(images[n])